## Preprocessing

In [215]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf
import numpy as np


#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("https://static.bc-edx.com/data/dla-1-2/m21/lms/starter/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [216]:
application_df["AFFILIATION"].value_counts()

Independent         18480
CompanySponsored    15705
Family/Parent          64
National               33
Regional               13
Other                   4
Name: AFFILIATION, dtype: int64

**Attempt 1 at Optimising - Combined few values as a one value in 'AFFILATION' column**

In [217]:
application_df["AFFILIATION"] = application_df["AFFILIATION"].replace(["Family/Parent","National","Regional"],"Other")
application_df["AFFILIATION"].value_counts()

Independent         18480
CompanySponsored    15705
Other                 114
Name: AFFILIATION, dtype: int64

In [218]:
application_df["ORGANIZATION"].value_counts() 

Trust           23515
Association     10255
Co-operative      486
Corporation        43
Name: ORGANIZATION, dtype: int64

**Attempt 2 at Optimising - Combined few values as a one value in 'ORGANIZATION' column**

In [219]:
application_df["ORGANIZATION"] = application_df["ORGANIZATION"].replace(["Association","Co-operative"],"Comb_Ass_Co-op")
application_df["ORGANIZATION"].value_counts() 

Trust             23515
Comb_Ass_Co-op    10741
Corporation          43
Name: ORGANIZATION, dtype: int64

In [220]:
application_df["STATUS"].value_counts()

1    34294
0        5
Name: STATUS, dtype: int64

In [221]:
application_df["SPECIAL_CONSIDERATIONS"].value_counts()

N    34272
Y       27
Name: SPECIAL_CONSIDERATIONS, dtype: int64

**Attempt 3 at Optimising - Remove extra two columns 'Status' and 'Special_Consideration'**

In [222]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
reduced_application_df = application_df.drop(columns=["EIN", "NAME","STATUS","SPECIAL_CONSIDERATIONS"])
reduced_application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,INCOME_AMT,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Comb_Ass_Co-op,0,5000,1
1,T3,Independent,C2000,Preservation,Comb_Ass_Co-op,1-9999,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Comb_Ass_Co-op,0,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,10000-24999,6692,1
4,T3,Independent,C1000,Heathcare,Trust,100000-499999,142590,1


In [223]:
# Determine the number of unique values in each column.
original_headings = reduced_application_df.columns.values.tolist()

reduced_application_df[original_headings].nunique()

APPLICATION_TYPE      17
AFFILIATION            3
CLASSIFICATION        71
USE_CASE               5
ORGANIZATION           3
INCOME_AMT             9
ASK_AMT             8747
IS_SUCCESSFUL          2
dtype: int64

In [224]:
# Look at APPLICATION_TYPE value counts for binning
reduced_application_df["APPLICATION_TYPE"].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T29        2
T15        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [225]:
# Choose a cutoff value and create a list of application types to be replaced
# use the variable name `application_types_to_replace`
application_types_to_replace = ['T9','T13','T12','T2','T25','T14','T29','T15','T17']

# Replace in dataframe
for app in application_types_to_replace:
    reduced_application_df['APPLICATION_TYPE'] = reduced_application_df['APPLICATION_TYPE'].replace(app,"Other")

# Check to make sure binning was successful
reduced_application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [226]:
# Look at CLASSIFICATION value counts for binning
reduced_application_df["CLASSIFICATION"].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C4120        1
C8210        1
C2561        1
C4500        1
C2150        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [227]:
# You may find it helpful to look at CLASSIFICATION value counts >1
reduced_application_df["CLASSIFICATION"].value_counts().loc[lambda x : x > 1]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C1800       15
C6000       15
C1250       14
C8200       11
C1238       10
C1278       10
C1235        9
C1237        9
C7210        7
C2400        6
C1720        6
C4100        6
C1257        5
C1600        5
C1260        3
C2710        3
C0           3
C3200        2
C1234        2
C1246        2
C1267        2
C1256        2
Name: CLASSIFICATION, dtype: int64

In [228]:
# Choose a cutoff value and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classifications_to_replace = reduced_application_df["CLASSIFICATION"].value_counts().loc[lambda x : x < 777].index.tolist()

# Replace in dataframe
for cls in classifications_to_replace:
    reduced_application_df['CLASSIFICATION'] = reduced_application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
reduced_application_df['CLASSIFICATION'].value_counts()


C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
Other     1484
C7000      777
Name: CLASSIFICATION, dtype: int64

In [229]:
reduced_application_df.dtypes

APPLICATION_TYPE    object
AFFILIATION         object
CLASSIFICATION      object
USE_CASE            object
ORGANIZATION        object
INCOME_AMT          object
ASK_AMT              int64
IS_SUCCESSFUL        int64
dtype: object

In [230]:
# Look at CLASSIFICATION value counts for binning
reduced_application_df['INCOME_AMT'].value_counts()

0                24388
25000-99999       3747
100000-499999     3374
1M-5M              955
1-9999             728
10000-24999        543
10M-50M            240
5M-10M             185
50M+               139
Name: INCOME_AMT, dtype: int64

**Attempt 4 at Optimising - Create a new bin to minimise the values of 'Income_Amt' column**

In [231]:
# Choose a cutoff value and create a list of Income_amt to be replaced
# use the variable name `income_amt_to_replace`

income_amt_to_replace = reduced_application_df["INCOME_AMT"].value_counts().loc[lambda x : x < 543].index.tolist()

# Replace in dataframe
for amt in income_amt_to_replace:
    reduced_application_df['INCOME_AMT'] = reduced_application_df['INCOME_AMT'].replace(amt,"above_5M")


income_amt_to_replace2 = ['1-9999','10000-24999','25000-99999']
for amt2 in income_amt_to_replace2:
    reduced_application_df['INCOME_AMT'] = reduced_application_df['INCOME_AMT'].replace(amt2,"1-99999")


# Check to make sure binning was successful
reduced_application_df['INCOME_AMT'].value_counts()

0                24388
1-99999           5018
100000-499999     3374
1M-5M              955
above_5M           564
Name: INCOME_AMT, dtype: int64

In [232]:
reduced_application_df['ASK_AMT'].describe()


count    3.429900e+04
mean     2.769199e+06
std      8.713045e+07
min      5.000000e+03
25%      5.000000e+03
50%      5.000000e+03
75%      7.742000e+03
max      8.597806e+09
Name: ASK_AMT, dtype: float64

**Attempt 5 at Optimising - Create 2 bins to include all values of ' ASK_AMT'**

In [202]:
reduced_application_df['ASK_AMT'] = np.where((reduced_application_df['ASK_AMT'] >= 4999) & (reduced_application_df['ASK_AMT'] <= 7750),'less than 7750',reduced_application_df['ASK_AMT'])


In [235]:
reduced_application_df.dtypes

APPLICATION_TYPE    object
AFFILIATION         object
CLASSIFICATION      object
USE_CASE            object
ORGANIZATION        object
INCOME_AMT          object
ASK_AMT              int64
IS_SUCCESSFUL        int64
dtype: object

In [236]:
reduced_application_df['ASK_AMT'].value_counts()

5000        25398
10478           3
15583           3
63981           3
6725            3
            ...  
5371754         1
30060           1
43091152        1
18683           1
36500179        1
Name: ASK_AMT, Length: 8747, dtype: int64

In [237]:
# Choose a cutoff value and create a list of ask_amt to be replaced
# use the variable name `ask_amt_to_replace`

ask_amt_to_replace = reduced_application_df["ASK_AMT"].value_counts().loc[lambda x : x < 25726].index.tolist()

# Replace in dataframe
for amt in ask_amt_to_replace:
    reduced_application_df['ASK_AMT'] = reduced_application_df['ASK_AMT'].replace(amt,"Greater than 7750")


# Check to make sure binning was successful
reduced_application_df['ASK_AMT'].value_counts()

Greater than 7750    34299
Name: ASK_AMT, dtype: int64

In [238]:
# Convert categorical data to numeric with `pd.get_dummies`
dummies_df = pd.get_dummies(reduced_application_df)
dummies_df

,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,APPLICATION_TYPE_T7,APPLICATION_TYPE_T8,...,USE_CASE_ProductDev,ORGANIZATION_Comb_Ass_Co-op,ORGANIZATION_Corporation,ORGANIZATION_Trust,INCOME_AMT_0,INCOME_AMT_1-99999,INCOME_AMT_100000-499999,INCOME_AMT_1M-5M,INCOME_AMT_above_5M,ASK_AMT_Greater than 7750
0,1,0,1,0,0,0,0,0,0,0,...,1,1,0,0,1,0,0,0,0,1
1,1,0,0,0,1,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,1
2,0,0,0,0,0,0,1,0,0,0,...,1,1,0,0,1,0,0,0,0,1
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,1,0,0,0,1
4,1,0,0,0,1,0,0,0,0,0,...,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34294,0,0,0,0,0,1,0,0,0,0,...,1,1,0,0,1,0,0,0,0,1
34295,0,0,0,0,0,1,0,0,0,0,...,1,1,0,0,1,0,0,0,0,1
34296,0,0,0,0,1,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,1
34297,1,0,0,0,0,0,1,0,0,0,...,1,1,0,0,1,0,0,0,0,1


In [239]:
# Split our preprocessed data into our features and target arrays
y = dummies_df.IS_SUCCESSFUL.values
X = dummies_df.drop(columns = "IS_SUCCESSFUL").values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=80, stratify=y)

In [240]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Compile, Train and Evaluate the Model

In [241]:
X_train

array([[0, 0, 0, ..., 0, 0, 1],
       [0, 0, 1, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       ...,
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1],
       [0, 0, 0, ..., 0, 0, 1]], dtype=uint8)

In [242]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  100
hidden_nodes_layer2 = 90
hidden_nodes_layer3 = 60
hidden_nodes_layer4 = 25


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Third hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer3, activation="relu"))

# Fourth hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer4, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_46 (Dense)            (None, 100)               3400      
                                                                 
 dense_47 (Dense)            (None, 90)                9090      
                                                                 
 dense_48 (Dense)            (None, 60)                5460      
                                                                 
 dense_49 (Dense)            (None, 25)                1525      
                                                                 
 dense_50 (Dense)            (None, 1)                 26        
                                                                 
Total params: 19,501
Trainable params: 19,501
Non-trainable params: 0
_________________________________________________________________


In [243]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

In [244]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=200)

Epoch 1/200
804/804 [==============================] - 3s 2ms/step - loss: 0.5672 - accuracy: 0.7235
Epoch 2/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5560 - accuracy: 0.7302
Epoch 3/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5528 - accuracy: 0.7308
Epoch 4/200
804/804 [==============================] - 3s 3ms/step - loss: 0.5515 - accuracy: 0.7310
Epoch 5/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5500 - accuracy: 0.7325
Epoch 6/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5494 - accuracy: 0.7313
Epoch 7/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5489 - accuracy: 0.7335
Epoch 8/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5479 - accuracy: 0.7328
Epoch 9/200
804/804 [==============================] - 2s 2ms/step - loss: 0.5476 - accuracy: 0.7332
Epoch 10/200
804/804 [==============================] - 2s 3ms/step - loss: 0.5475 - accura

In [246]:
# Evaluate the model using the test data
model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5828 - accuracy: 0.7259 - 373ms/epoch - 1ms/step
Loss: 0.5827805399894714, Accuracy: 0.7259474992752075


In [247]:
# Export our model to HDF5 file
nn.save('AlphabetSoupCharity_Optimisation.h5')


In [248]:
from google.colab import files
files.download('AlphabetSoupCharity_Optimisation.h5')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>